In [1]:
from keras.models import Model
from keras.layers import *
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau,LearningRateScheduler
import scipy.io
import numpy as np
from Utils import *
from keras import backend as K
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
#### load seismic data

def loadx():
    x = []
    for i in range(1,30):
        path = "./Data/FORGE/Train/Noisy/Noisy_{}.mat".format(i)
        data = scipy.io.loadmat(path)
        x1=data['dn']
        x2 = yc_patch(x1, w1, w2, z1, z2)
        x.append(x2)
    return x

def loady():
    y = []
    for i in range(1,30):
        path = "./Data/FORGE/Train/Clean/Clean_{}.mat".format(i)
        data = scipy.io.loadmat(path)
        y1=data['d']
        y2=yc_patch(y1,w1,w2,z1,z2)
        y.append(y2)
    return y

In [3]:
### generate datasets

w1=64
w2=64
z1=32
z2=32

X = loadx()
Y = loady()

X = np.array(X)
Y = np.array(Y)

X = np.reshape(X,(X.shape[0]*X.shape[1], w1, w2,1))
Y = np.reshape(Y,(Y.shape[0]*Y.shape[1], w1, w2,1))
print(X.shape)

Xtrain, Xval, Ytrain, Yval = train_test_split(X, Y, test_size=0.2, random_state=41)

(52142, 64, 64, 1)


In [4]:
ker =4
dropout=0.1
stride=1
layer=2
r=8
m=2
L=32

input_dims = (w1,w2,1)
input_img = Input(shape=input_dims)

def SLKNet():
    C1 = Conv2D(ker, 3, padding='same', strides=1)(input_img)
    C1 = BatchNormalization()(C1)
    C1 = Activation('relu')(C1)
# -------------------------Encoder Part--------------------------#

    DB1 = DenseBlock(C1, layer, ker*2, dropout=dropout)
    SK1 = SK(DB1, m=m, r=r, L=L, kernel=ker*2)
    CB1 = BlockA(SK1,ker*2,dropout=dropout)
    TD1 = TD(SK1,ker*2,stride)

    DB2 = DenseBlock(TD1, layer, ker*4,dropout= dropout)
    SK2 = SK(DB2, m=m, r=r, L=L, kernel=ker*4)
    CB2 = BlockA(SK2,ker*4,dropout=dropout)
    TD2 = TD(SK2, ker*4,stride)

    DB3 = DenseBlock(TD2, layer, ker*8,dropout= dropout)
    SK3 = SK(DB3, m=m, r=r, L=L ,kernel=ker*8)
    CB3 = BlockA(SK3, ker*8, dropout=dropout)
    TD3 = TD(SK3,ker*8,stride)

    DB6 = DenseBlock(TD3, layer, ker*16, dropout=dropout)

    lab = TU(DB6,stride, ker*8, dropout=dropout)
    lab = DenseBlock(lab,layer, ker*8, dropout=dropout)
    lab = SK(lab, m=m, r=r, L=L, kernel= ker*8)
    lab = concatenate([lab,CB3])

    lab = TU(lab,stride,ker*4, dropout=dropout)
    lab = DenseBlock(lab,layer,ker*4, dropout=dropout)
    lab = SK(lab, m=m, r=r, L=L, kernel=ker*4)
    lab = concatenate([lab,CB2])

    lab = TU(lab,stride,ker*2, dropout=dropout)
    lab = DenseBlock(lab,layer,ker*2, dropout=dropout)
    lab = SK(lab, m=m, r=r, L=L, kernel=ker*2)
    lab = concatenate([lab,CB1])

    lab = Conv2D(1, 3, padding = 'same', strides=1, activation='linear',name='outlab')(lab)

    model = Model(input_img, outputs=lab)
    model.summary()

    model.compile(optimizer='adam', loss='mse')
    return model

model = SLKNet()

Instructions for updating:
Colocations handled automatically by placer.
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 64, 64, 4)    40          ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 64, 64, 4)   16          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activatio

                                                                                                  
 lambda_1 (Lambda)              (None, 64, 64, 8)    0           ['multiply[0][0]']               
                                                                                                  
 conv2d_7 (Conv2D)              (None, 64, 64, 8)    584         ['lambda_1[0][0]']               
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 64, 64, 8)   32          ['conv2d_7[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 activation_7 (Activation)      (None, 64, 64, 8)    0           ['batch_normalization_7[0][0]']  
                                                                                                  
 conv2d_8 

                                                                                                  
 batch_normalization_14 (BatchN  (None, 64, 64, 16)  64          ['conv2d_14[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_14 (Activation)     (None, 64, 64, 16)   0           ['batch_normalization_14[0][0]'] 
                                                                                                  
 conv2d_15 (Conv2D)             (None, 64, 64, 32)   4640        ['activation_14[0][0]']          
                                                                                                  
 batch_normalization_15 (BatchN  (None, 64, 64, 32)  128         ['conv2d_15[0][0]']              
 ormalization)                                                                                    
          

 activation_21 (Activation)     (None, 64, 64, 32)   0           ['batch_normalization_21[0][0]'] 
                                                                                                  
 conv2d_22 (Conv2D)             (None, 64, 64, 64)   18496       ['activation_21[0][0]']          
                                                                                                  
 batch_normalization_22 (BatchN  (None, 64, 64, 64)  256         ['conv2d_22[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_22 (Activation)     (None, 64, 64, 64)   0           ['batch_normalization_22[0][0]'] 
                                                                                                  
 dropout_9 (Dropout)            (None, 64, 64, 64)   0           ['activation_22[0][0]']          
          

                                                                                                  
 global_average_pooling2d_7 (Gl  (None, 32)          0           ['activation_29[0][0]']          
 obalAveragePooling2D)                                                                            
                                                                                                  
 add_3 (Add)                    (None, 32)           0           ['global_average_pooling2d_6[0][0
                                                                 ]',                              
                                                                  'global_average_pooling2d_7[0][0
                                                                 ]']                              
                                                                                                  
 dense_6 (Dense)                (None, 32)           1056        ['add_3[0][0]']                  
          

 ormalization)                                                                                    
                                                                                                  
 batch_normalization_35 (BatchN  (None, 64, 64, 16)  64          ['conv2d_33[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_34 (Activation)     (None, 64, 64, 16)   0           ['batch_normalization_34[0][0]'] 
                                                                                                  
 activation_35 (Activation)     (None, 64, 64, 16)   0           ['batch_normalization_35[0][0]'] 
                                                                                                  
 global_average_pooling2d_8 (Gl  (None, 16)          0           ['activation_34[0][0]']          
 obalAvera

                                                                                                  
 batch_normalization_39 (BatchN  (None, 64, 64, 8)   32          ['conv2d_36[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_39 (Activation)     (None, 64, 64, 8)    0           ['batch_normalization_39[0][0]'] 
                                                                                                  
 conv2d_37 (Conv2D)             (None, 64, 64, 8)    584         ['activation_39[0][0]']          
                                                                                                  
 conv2d_38 (Conv2D)             (None, 64, 64, 8)    1608        ['activation_39[0][0]']          
                                                                                                  
 batch_nor

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('best_model_forge.h5', monitor='val_loss', mode='min', save_best_only=True)

batch = 128

model.fit(Xtrain,Ytrain, batch_size=batch, verbose=1,epochs=100,callbacks=[es, mc],validation_data=(Xval,Yval))